### Import packages and get access to the training text file

In [1]:
### For colab usage
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive/Colab Notebooks/6334/HW4

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/6334/HW4


In [2]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.2 MB/s 


In [ ]:
import unidecode
import string
import random
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os

from tqdm import tqdm

In [ ]:
### helpers.py

def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file, len(file)

filename = 'medline.0.txt'
file, file_len = read_file(filename)

### Functions to create the model

In [ ]:
### train.py
### corrected one mistake in train of using cuda
### corrected one mistake in the last line of train: previous code is loss.data[0] 
### which causes error, so changed to loss

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    if cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(batch_size)

    if cuda: 
        #Can't convert hidden to cuda because hidden is a tuple of tensor, not tensor. 
        #Need to convert it to list, then convert each of the elements to cuda,
        #then convert back to a tuple.
        hidden = list(hidden)
        hidden[0] = hidden[0].cuda()
        hidden[1] = hidden[1].cuda()
        hidden = tuple(hidden)
        #hidden = hidden.cuda()

    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss / chunk_len

def save():
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

In [ ]:
### model.py

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
            #self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, bias=False) #try to set bias as False
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
### generate.py
### corrected one mistake of using cuda, the same as in train

def generate(decoder, prime_str='A', predict_len=100, temperature=0.8, cuda=False):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    if cuda:
        #Can't convert hidden to cuda because hidden is a tuple of tensor, not tensor. 
        #Need to convert it to list, then convert each of the elements to cuda,
        #then convert back to a tuple.
        hidden = list(hidden)
        hidden[0] = hidden[0].cuda()
        hidden[1] = hidden[1].cuda()
        hidden = tuple(hidden)
        #hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
        
    inp = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        if cuda:
            inp = inp.cuda()

    return predicted

In [ ]:
### healpers.py

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

### Training and evaluation

In [ ]:
all_characters = string.printable
n_characters = len(all_characters)
n_layers = 2
n_epochs = 2000

hidden_size = 100
batch_size = 100
learning_rate = 0.01
model = "lstm"

chunk_len = 500
print_every = 100
cuda = True

In [ ]:
decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=model,
    n_layers=n_layers,
)

if cuda:
    decoder.cuda()

decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

In [ ]:
for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set(chunk_len, batch_size))
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(generate(decoder, 'Wh', 100, cuda=cuda), '\n')

print("Saving...")
save()

  5%|▌         | 100/2000 [02:54<57:06,  1.80s/it] 

 54s (100 5%) 1.7931]
Whanas ang the and in tumury weras in thor. Herkerk (TM) apsine recermes (
SB  - BACF (TQA The in scum 



 10%|█         | 200/2000 [05:46<51:53,  1.73s/it]

 46s (200 10%) 1.4480]
Whytics
MH  - Adult
MH  - Ordags
MH  - Contite Skuakizations D
AU  - Brainei A
FAU - Recang, F
AU  - L 



 15%|█▌        | 300/2000 [08:43<49:31,  1.75s/it]  

...

 38s (1800 90%) 0.9610]
Whe directs analysis in vistem. METHOD: Bacterial oxidative in some Kapea 
      OS or self-care in ga 



 95%|█████████▌| 1900/2000 [55:33<02:56,  1.76s/it]

 33s (1900 95%) 0.9823]
Wha indicates for negative statistically sample. The improve approaches of the however and medical 
   



100%|██████████| 2000/2000 [58:25<00:00,  1.75s/it]

 25s (2000 100%) 1.0069]
Whar C
FIR - Cattogo, Andrie
IR  - Pertad G
FIR - Roserenst, Rean
IR  - Park A
FIR - Delberg, Taka
AU  

Saving...
Saved as medline.0.pt


In [ ]:
generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature=0.8, cuda=True)

'PMID- 23017638\nOWN - NLM\nSTAT- MEDLINE\nDCOM- 20130313\nLR  - 20171125\nIS  - 1568-7142 (Electronic)\nIS  - 0009-4963 (Linking)\nVI  - 143\nIP  - 1\nDP  - 2013 Feb\nTI  - The simulate of\n      limited the simulating number of the progression and positive\n      not assays, demethy, its receptor response to the increase of CAT, use of the mice.\nPG  - 697-647 CPHD-100254241005 [pii]\nAID - 10.1158/1078-01007-1-123. Epub 2012 Sep 27.\n\nPMID- 22989364\nOWN - NLM\nSTAT- MEDLINE\nDCOM- 20130521\nLR  - 20171108\nIS  - 1539-3627 (Electronic)\nIS  - 0742-1865 (Linking)\nVI  - 18\nIP  - 11\nDP  - 2012 Dec\nTI  - Cancer carbold who various cell lines in Biol directly that\n      someter for helpyly cell cancer and 10 healthy show use of RATC3 delivery of\n      the cisplatin-casurgery therapy of study, we should be the diagnosis\n      interactions, and has \n      with cancer health targeting the pathways and\n      performed to level from large discus symptom in human PSA (3041) and we to